In [16]:
%loadFromPOM pom.xml

In [17]:
import ca.uhn.fhir.context.FhirContext;
import ca.uhn.fhir.rest.client.api.*;
import ca.uhn.fhir.rest.api.MethodOutcome;
import org.hl7.fhir.r4.model.*;

In [18]:
FhirContext ctx = FhirContext.forR4();
String serverBase = "http://localhost:8080/fhir";
IGenericClient client = ctx.newRestfulGenericClient(serverBase);

Create a FHIR Observation resource
Observation.category:

coding.system - http://hl7.org/fhir/observation-category
coding.code - vital-signs
So we were right with our category configuration. Now let's find out the code value.

 Observation.code:
 coding.system - http://loinc.org
 coding.code - one of the allowed vital sign result types, in our case it is 8310-5 for body temperature, 
               8480-6 for systolic blood pressure and 8462-4 for diastolic blood pressure.
 
 So we were right with our code configuration. Now let's find out the value.
 Observation.valueQuantity:
 value - the actual value of the vital sign measurement, in our case it is 96.8 for body temperature,
 unit - the unit of measurement, in our case it is °C for body temperature, mmHg for blood pressure
 system - http://unitsofmeasure.org
 code - the code for the unit of measurement, in our case it is °C for body temperature, mm[Hg] for blood pressure
 
 So we were right with our valueQuantity configuration. Now let's find out the effectiveDateTime value.
 Observation.effectiveDateTime:
 effectiveDateTime - the date and time the measurement was taken, in our case it is 2021-09-01T12:35:00+03:00

In [19]:
Observation observation = new Observation();
observation.addCategory()
    .addCoding()
    .setSystem("http://hl7.org/fhir/observation-category")
    .setCode("vital-signs");
observation.getCode()
    .addCoding()
    .setSystem("http://loinc.org")
    .setCode("8310-5");
observation.setValue(
    new Quantity()
        .setValue(96.8)
        .setUnit("degF")
        .setSystem("http://unitsofmeasure.org")
        .setCode("Cel"));
observation.setEffective(new DateTimeType("2021-09-01T12:35:00+03:00"));
observation.setStatus(Observation.ObservationStatus.FINAL);

org.hl7.fhir.r4.model.Observation@58f4b0e0

In [ ]:
Patient patient = new Patient();
Bundle result = client.search()
            .forResource(Patient.class)
            .where(Patient.FAMILY.matches().value("陳")).and(Patient.GIVEN.matches().value("大明"))
            .returnBundle(Bundle.class)
            .execute();
patient = (Patient) result.getEntry().get(0).getResource();
System.out.println(patient.getId());

Patient reference is stored in Observation.subject  
儲存Observation資料 

In [ ]:
observation.setSubject(new Reference("Patient/" + patient.getIdElement().getIdPart()));
// Save the observation
MethodOutcome outcome = client.create()
    .resource(observation)
    .prettyPrint()
    .encodedJson()
    .execute();
System.out.println("Created Observation, ID: " + outcome.getId().getValue());